In [1]:
# importing modules

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import logging
import sys
import pickle
import joblib
import os
import gc

from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization

from tensorflow.keras.regularizers import l1, l2

from tensorflow.python.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.test.gpu_device_name()
tf.device('/GPU:0')

import optuna
from optuna.samplers import TPESampler

2024-11-23 16:15:02.808588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 16:15:03.091742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732378503.224848   10142 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732378503.257172   10142 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 16:15:03.550109: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
num_classes = 19 #0 - 18
np.random.seed(441)
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

def dataset(X, Y):
    imgs = []
    labels = []

    for _ in range(len(X)):
        i, j = np.random.choice(len(X), size=2, replace=False)
        img1, img2 = X[i], X[j]
        combined_img = np.hstack((img1, img2))
        combined_label = Y[i] + Y[j]
        imgs.append(combined_img)
        labels.append(combined_label)
    return np.array(imgs), np.array(labels)

x_train, y_train = dataset(x_train, y_train)
x_test, y_test = dataset(x_test, y_test)

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 56)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [ ]:
# for i in range(5):
#     plt.imshow(x_train[i])
#     plt.show()
#     print(np.where(y_train[i]==1)[0][0])

In [4]:
# s_name = "nn_study"
# m_name = "nn_model"

# def objective(trial):
#     learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-4, log=True)
#     n_neurons = trial.suggest_int('n_neurons', 500, 600)
#     dropout_rate = trial.suggest_float('dropout_rate', 0.0001, 0.001)
#     l1_regularization = trial.suggest_float('l1_regularization', 1e-5, 1e-4)
#     n_layers = trial.suggest_int('n_layers', 5, 6)
    
#     model = Sequential()
#     model.add(Flatten(input_shape=(56, 28)))

#     for _ in range(n_layers):
#         model.add(Dense(n_neurons, activation='relu', kernel_regularizer=l1(l1_regularization)))
#         model.add(Dropout(dropout_rate))

#     model.add(Dense(num_classes,activation='softmax'))

#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

#     # early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1, start_from_epoch=10)
#     # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
#     model_checkpoint = ModelCheckpoint(f"best_{m_name}.keras", save_best_only=True, monitor='val_loss', mode='min', verbose=0)

#     model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15,
#                         batch_size=32, verbose=0, callbacks=[model_checkpoint])
    
#     score = model.evaluate(x_test, y_test, verbose=0)
#     accuracy = score[1]

#     backend.clear_session()
#     del model
#     gc.collect()

#     return accuracy

# study = optuna.create_study(direction='maximize', study_name=s_name, storage=f"sqlite:///{s_name}.db", load_if_exists=True, sampler=TPESampler(seed=441))

# study.optimize(objective, n_trials=13, show_progress_bar=True)
# joblib.dump(study, f"{s_name}.pkl")

[I 2024-11-23 16:16:00,098] Using an existing study with name 'nn_study' instead of creating a new one.


  0%|          | 0/13 [00:00<?, ?it/s]

/home/ljf1/dis/disvenv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1732378565.236400   10317 service.cc:148] XLA service 0x7f7cb401ae80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732378565.236765   10317 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti Laptop GPU, Compute Capability 8.6
2024-11-23 16:16:05.301607: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732378565.688378   10317 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-23 16:16:08.440489: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warnin

[I 2024-11-23 16:18:16,981] Trial 35 finished with value: 0.9039999842643738 and parameters: {'learning_rate': 7.419892844138638e-05, 'n_neurons': 553, 'dropout_rate': 0.00027430010041840823, 'l1_regularization': 8.66456631389732e-05, 'n_layers': 5}. Best is trial 14 with value: 0.913100004196167.


2024-11-23 16:18:34.333051: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 116 bytes spill stores, 116 bytes spill loads

2024-11-23 16:18:34.345509: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_169', 116 bytes spill stores, 116 bytes spill loads



...

2024-11-23 16:39:52.553090: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_147', 116 bytes spill stores, 116 bytes spill loads

2024-11-23 16:39:52.594670: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_86', 116 bytes spill stores, 116 bytes spill loads

2024-11-23 16:39:55.220886: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_147', 152 bytes spill stores, 160 bytes spill loads

2024-11-23 16:39:55.264921: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_86', 152 bytes spill stores, 160 bytes spill loads



[I 2024-11-23 16:41:44,235] Trial 46 finished with value: 0.8907999992370605 and parameters: {'learning_rate': 6.0502067395865816e-05, 'n_neurons': 593, 'dropout_rate': 0.00024630872645750503, 'l1_regularization': 5.001149182651993e-05, 'n_layers': 5}. Best is trial 14 with value: 0.913100004196167.


2024-11-23 16:41:57.928612: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_86', 36 bytes spill stores, 36 bytes spill loads

2024-11-23 16:41:58.071476: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_86', 272 bytes spill stores, 272 bytes spill loads

2024-11-23 16:41:58.350448: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_147', 604 bytes spill stores, 624 bytes spill loads

2024-11-23 16:41:58.358333: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_86', 444 bytes spill stores, 480 bytes spill loads

2024-11-23 16:41:58.392237: I external/local_xla/xla/stream_e

[I 2024-11-23 16:43:50,219] Trial 47 finished with value: 0.9063000082969666 and parameters: {'learning_rate': 7.301249198128112e-05, 'n_neurons': 588, 'dropout_rate': 0.00014921225387433238, 'l1_regularization': 7.429417533380305e-05, 'n_layers': 5}. Best is trial 14 with value: 0.913100004196167.


['nn_study.pkl']

In [5]:
# print('Best trial:')
# print(f' Value: {study.best_trial.value}')
# print(' Params: ')
# for key, value in study.best_trial.params.items():
#     print(f'    {key}: {value}')

Best trial:
 Value: 0.913100004196167
 Params: 
    learning_rate: 9.999014745181874e-05
    n_neurons: 566
    dropout_rate: 0.0002658695997620067
    l1_regularization: 4.540153304979403e-05
    n_layers: 6


In [6]:
# def create_and_save_model(study):
#     with open("sampler.pkl", "wb") as fout:
#         pickle.dump(study.sampler, fout)
        
#     params = study.best_trial.params
#     model = Sequential()
#     model.add(Flatten(input_shape=(56, 28)))
#     print()
#     for _ in range(params.get('n_layers')):
#         model.add(Dense(params.get('n_neurons'), activation='relu', kernel_regularizer=l1(params.get('l1_regularization'))))
#         model.add(Dropout(params.get('dropout_rate')))

#     model.add( Dense(num_classes,activation='softmax'))

#     model.compile(optimizer=Adam(params.get('learning_rate')), loss='categorical_crossentropy', metrics=['accuracy'])

#     early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

#     history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50,
#                         batch_size=128, verbose=1)

#     score = model.evaluate(x_test, y_test, verbose=0)
#     accuracy = score[1]
#     print(accuracy)
#     model.save(f'{m_name}.keras')
#     with open('nn_history.pkl', "wb") as file:
#         pickle.dump(history.history, file)

# create_and_save_model(study)


Epoch 1/50
449/469 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1892 - loss: 5.6829

2024-11-23 16:44:00.514681: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2772', 36 bytes spill stores, 36 bytes spill loads

2024-11-23 16:44:00.682487: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2772', 36 bytes spill stores, 36 bytes spill loads



469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1952 - loss: 5.6519

2024-11-23 16:44:04.891499: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 28 bytes spill stores, 28 bytes spill loads

2024-11-23 16:44:04.952995: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 312 bytes spill stores, 324 bytes spill loads

2024-11-23 16:44:05.378935: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90_0', 12 bytes spill stores, 12 bytes spill loads

2024-11-23 16:44:05.400460: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90_0', 840 bytes spill stores, 840 bytes spill loads

2024-11-23 16:44:05.512738: I external/local_xla/xla/stream_

469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.1955 - loss: 5.6502 - val_accuracy: 0.6033 - val_loss: 3.6782
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6658 - loss: 3.4312 - val_accuracy: 0.7566 - val_loss: 3.0190
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7897 - loss: 2.8981 - val_accuracy: 0.8126 - val_loss: 2.7605
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8437 - loss: 2.6518 - val_accuracy: 0.8322 - val_loss: 2.6165
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8732 - loss: 2.4776 - val_accuracy: 0.8523 - val_loss: 2.4871
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8966 - loss: 2.3360 - val_accuracy: 0.8558 - val_loss: 2.4049
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9116 - loss: 2.2210 - val_accuracy: 0.8674 - val_loss: 2.3151
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9267 - loss: 2.1133 - val_accuracy: 0.8619 - va

2024-11-23 16:45:24.137556: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 28 bytes spill stores, 28 bytes spill loads

2024-11-23 16:45:24.370420: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 288 bytes spill stores, 292 bytes spill loads

2024-11-23 16:45:24.679808: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 668 bytes spill stores, 680 bytes spill loads

2024-11-23 16:45:24.878755: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_90', 216 bytes spill stores, 216 bytes spill loads

2024-11-23 16:45:24.942944: I external/local_xla/xla/stream_ex

0.9002000093460083


In [7]:
model = load_model(f'{m_name}.keras')
sampler = pickle.load(open("sampler.pkl", "rb"))
history = pickle.load(open("nn_history.pkl", "rb"))

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 1568)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 566)            │       888,054 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 566)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 566)            │       320,922 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 566)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 566)            │       320,922 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 566)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 566)            │       320,922 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 566)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 566)            │       320,922 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 566)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 566)            │       320,922 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 566)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 19)             │        10,773 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,510,313 (28.65 MB)

 Trainable params: 2,503,437 (9.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,006,876 (19.10 MB)

In [9]:
mask = (model.predict(np.array(x_test).reshape((10000, 28*56))) != y_test)
x_false = x_test[mask]

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 1568), dtype=float32). Expected shape (None, 56, 28), but input has incompatible shape (32, 1568)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 1568), dtype=float32)
  • training=False
  • mask=None

In [ ]:
print(x_test.shape)  # Should be (n_samples, n_features)
print(y_test.shape)  # Should be (n_samples,) for classification or (n_samples, n_labels) for multi-label tasks
predictions = model.predict(x_test)
print(predictions.shape)  # Check the shape of the predicted outputs


In [ ]:
print(model.predict(np.array(x_test))[:5])
print(y_test[:5])